In [23]:
import os
import sqlite3
import schedule
import time

import re

In [3]:
# os.system("ansible-playbook run_model.yaml -t pred > predlog.txt")

rows = []
p_start = re.compile('Prediction :.+')
p_end = re.compile('PLAY RECAP.+')

with open ("predlog.txt", 'r') as f:
    lines = f.readlines()

    for line in lines:
        start = p_start.search(str(line))

        if start:
            idx = lines.index(line)
            run = True

            while run:
                ## drop no-useful strings and characters
                print(lines[idx])
                lines[idx] = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s\.]", "", lines[idx])
                lines[idx] = re.sub("\n", "", lines[idx])
                lines[idx] = re.sub("n\d+", "", lines[idx])

                rows.append(lines[idx])
                ## search end of task
                end = p_end.search(lines[idx])

                idx += 1

                if end:
                    run = False
                    
print(rows)

        "Prediction :  [('n02219486', 'ant', 0.9818205), ('n02165105', 'tiger_beetle', 0.00959373), ('n02168699', 'long-horned_beetle', 0.0039857603), ('n02268443', 'dragonfly', 0.00078227365), ('n02229544', 'cricket', 0.00072638784)]"

    ]

}



PLAY RECAP *********************************************************************

['        Prediction    ant 0.9818205  tigerbeetle 0.00959373  longhornedbeetle 0.0039857603  dragonfly 0.00078227365  cricket 0.00072638784', '    ', '', '', 'PLAY RECAP ']


In [7]:
print(rows[0].split())

['Prediction', 'ant', '0.9818205', 'tigerbeetle', '0.00959373', 'longhornedbeetle', '0.0039857603', 'dragonfly', '0.00078227365', 'cricket', '0.00072638784']


In [18]:
data = rows[0].split()

print(data)

['Prediction', 'ant', '0.9818205', 'tigerbeetle', '0.00959373', 'longhornedbeetle', '0.0039857603', 'dragonfly', '0.00078227365', 'cricket', '0.00072638784']


In [19]:
data = data[1:]

print(data)

['ant', '0.9818205', 'tigerbeetle', '0.00959373', 'longhornedbeetle', '0.0039857603', 'dragonfly', '0.00078227365', 'cricket', '0.00072638784']


In [20]:
cls = data[0::2]
data = data[1::2]

In [21]:
print(cls)

['ant', 'tigerbeetle', 'longhornedbeetle', 'dragonfly', 'cricket']


In [22]:
print(data)

['0.9818205', '0.00959373', '0.0039857603', '0.00078227365', '0.00072638784']


In [15]:
nodes = []

## load current nodes info from hosts table
con = sqlite3.connect('../nodes.db3')
cur = con.cursor()
cur.execute('select name from nodes where type = "builder"')
tmp = cur.fetchall()

## save node lists
for node in tmp:
    node = list(node)
    node = str(node)
    node = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", node)
    nodes.append(node)
    
print(nodes)

['rpi6401']


In [24]:
now = round(time.time())
log = []

for i in range(len(cls)):
    tmp = []
    tmp.append(now)
    tmp.append(nodes[0])
    tmp.append(cls[i])
    tmp.append(data[i])
    tmp = tuple(tmp)
    log.append(tmp)

In [25]:
print(log)

[(1673595515, 'rpi6401', 'ant', '0.9818205'), (1673595515, 'rpi6401', 'tigerbeetle', '0.00959373'), (1673595515, 'rpi6401', 'longhornedbeetle', '0.0039857603'), (1673595515, 'rpi6401', 'dragonfly', '0.00078227365'), (1673595515, 'rpi6401', 'cricket', '0.00072638784')]


In [26]:
## insert data into DB
con = sqlite3.connect('../nodes.db3')
cur = con.cursor()
query = "insert into modelpred values(?,?,?,?);"
cur.executemany(query, log)
con.commit()

## show result
cur.execute('select * from modelpred')
out = cur.fetchall()

for col in out:
    print(col)

con.close()

(1673595515, 'rpi6401', 'ant', '0.9818205')
(1673595515, 'rpi6401', 'tigerbeetle', '0.00959373')
(1673595515, 'rpi6401', 'longhornedbeetle', '0.0039857603')
(1673595515, 'rpi6401', 'dragonfly', '0.00078227365')
(1673595515, 'rpi6401', 'cricket', '0.00072638784')
